In [7]:
import json
import os
from opencc import OpenCC
import requests

# 读取并返回 JSON 数据
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)
    
# 格式化 JSON 数据为字符串
def format_json(data):
    return json.dumps(data, ensure_ascii=False, indent=4)

# 从指定 URL 获取 JSON 并返回解析后的 Python 对象
def fetch_json(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

In [8]:
tw_url = 'https://sg-tools-cdn.blablalink.com/hw-20/xg-89/00797de77cede337d41d30c45aadb122.json'
en_url = 'https://sg-tools-cdn.blablalink.com/yl-57/hd-03/1bf030193826e243c2e195f951a4be00.json'


tw_data = fetch_json(tw_url)
en_data = fetch_json(en_url)

en_dict = { entry['id']: entry for entry in en_data }

cc = OpenCC('t2s')  # 将繁体转换为简体

combined_list = []
for tw in tw_data:
    uid = tw['id']
    if uid not in en_dict:
        # 如果英文表里没有对应的 id，可以选择跳过或保留部分字段；这里跳过
        continue

    en = en_dict[uid]

    # 提取各字段
    item = {
        'id': uid,
        'name_code': tw.get('name_code'),
        'class': tw.get('class'),
        # 将繁体中文转换为简体
        'name_cn': cc.convert( tw['name_localkey']['name'] ),
        'name_en': en['name_localkey']['name'],
        'element': tw['element_id']['element']['element'],
        'use_burst_skill': tw.get('use_burst_skill'),
        'corporation': tw.get('corporation'),
        'weapon_type': tw['shot_id']['element']['weapon_type'],
        'original_rare': tw.get('original_rare'),
    }
    combined_list.append(item)


output_path = 'nikke_list.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(combined_list, f, ensure_ascii=False, indent=4)